# Computational psychrometric analysis of cooling systems as a control problem

**Note**: Chapter numbers correspond to the accompaning paper:

> C. Ghiaus (2021) Computational psychrometric analysis of cooling systems as a control problem, International Journal of Building Performance Simulation


Model: 
- Cooling coil with by-pass, reheating coil, thermal zone, mixing recycled and outdoor air (see *lin_model* and *solve_lin* in `cool.py`).

Types of systems (see `cool.py`):
- CAV: constant air volume - the mass flow rate is constant (see *CAV_wd* in `cool.py`).
- VBP: CAV system with variable mix-air bypass (see *VBP_wd* in `cool.py`). 
- VAV: variable air volume - the mass flow rate is variable (see *VAV_wd* in `cool.py`).

**Types of controls** (numbers correspond to the sections in Ghiaus (2021):

Linear control problem: constant air volume (CAV) systems

    5.1 Control of indoor air temperature & humidity in CAV systems with reheating
    5.2 Control of indoor air temperature in CAV systems without reheating


Non-linear parameter optimization problem

    6.1 By-pass control in constant air volume (CAV) systems without reheating
    
        6.1.1. Control of indoor air temperature & humidity in CAV systems
        6.1.2. Control of indoor and supply air temperatures in CAV systems
    
    6.2 Mass-flow control in variable air volume (VAV) systems
    
        6.2.1 Control of indoor air temperature & humidity in VAV systems without reheating
        6.2.2 Control of indoor and supply air temperatures in VAV systems without reheating
        6.2.3 Control of indoor air temperature & humidity and of supply air temperature in VAV systems with reheating

In [1]:
import ipywidgets as wd
import matplotlib.pyplot as plt
# %matplotlib inline  # uncomment for inline figure
# %matplotlib qt      # uncomment for figure in separate window
# plt.show()

plt.rcParams["figure.figsize"] = (10, 7.7)
font = {'size': 16}
plt.rc('font', **font)

In [2]:
import cool as cc

QsTZ =  45,000.0 W
QlTZ =  15,000.0 W


### Create air handing unit (AHU) object

In [3]:
Kθ, Kw = 1e10, 0        # Gain factors of the P-controllers
β = 0.16                # By-pass factor of the cooling coil

m, mo = 3.1, 1.         # kg/s, mass flow rate, supply and outdoor air
θo, φo = 32., 0.5       # °C, -, outdoor air temperature and humidity
θ5sp, φ5sp = 26., 0.5   # °C, -, indoor air temperature and humidity set points

mi = 1.35               # kg/s, mass flow rate of infiltration air
UA = 675.               # W/K, overall heat transfet coeffcient
QsBL, QlBL = 34000., 4000.    # W, sensible & latent auxiliar heat

parameters = m, mo, β, Kθ, Kw
inputs = θo, φo, θ5sp, φ5sp, mi, UA, QsBL, QlBL

# 5 Linear control problem: constant air volume (CAV) systems

## 5.1 Control of indoor air temperature & humidity in CAV systems with reheating

<img src="Figures/Legend.png" alt="Legend_figures" style="width: 600px;"/>

This legend is for all figures.

<img src="Figures/CAV_θ5_φ5.png" alt="CAV_θ5_φ5" style="width: 600px;"/>

> CAV system ontroling indoor temperature, $\theta_5$, and humidity, $\varphi_5$, by acting on the cooling coil [CC], $\dot{Q}_{tCC}$, and heating coil [HC],  $\dot{Q}_{sHC}$, respectively.
> - The mass flow rate of supply air, $\dot{m}$, is given and constant (CAV system).
> - The supply air temperature, $\theta_4$, is not controlled.

In [4]:
cool0 = cc.MxCcRhTzBl(parameters, inputs)
Kw = 1e10
cool0.actual[4] = Kw
wd.interact(cool0.CAV_wd, θo=(26, 34), φo=(0.4, 1), θ5sp=(20, 28), φ5sp=(0.30, 1, 0.01),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), QsBL=(0, 60_000, 500), QlBL=(0, 20_000, 500))

interactive(children=(IntSlider(value=32, description='θo', max=34, min=26), FloatSlider(value=0.7, descriptio…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

## 5.2 Control of indoor air temperature in CAV systems without reheating ($K_w = 0$)

<img src="Figures/CAV_θ5.png" alt="CAV_θ5" style="width: 600px;"/>

> CAV system controling the indoor temperature, $\theta_5$, by acting on the cooling coil [CC], $\dot{Q}_{tCC}$.
> - The heating coil is not operating, $\dot{Q}_{sHC}=0$. Indoor humidity, $w_5$, is not controlled.
> - The mass flow rate of supply air, $\dot{m}$, is given and constant (CAV system).
> - The supply air temperature, $\theta_4 = \theta_3$, is not controlled.

*Note on widgets*: 
- Acting on the set point of indoor humidity, $\varphi_{5sp}$ = `φ5sp`, has no effect.

In [5]:
cool1 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool1.CAV_wd, θo=(26, 34), φo=(0.4, 1), θ5sp=(20, 28), φI5sp=(0.4, 1, 0.01),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), QsBL=(0, 60_000, 500), QlBL=(0, 20_000, 500))

interactive(children=(IntSlider(value=32, description='θo', max=34, min=26), FloatSlider(value=0.7, descriptio…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

# 6 Non-linear parameter optimization problem

## 6.1 By-pass control in constant air volume (CAV) systems without reheating

### 6.1.1 Control of indoor air temperature & humidity in CAV systems

<img src="Figures/CAV_β_θ5_φ5.png" alt="CAV_β_θ5_φ5" style="width: 600px;"/>

> CAV with mix-air bypass controlling: indoor temperature, $\theta_5$, by acting on the cooling coil [CC], $\dot{Q}_{tCC}$, and the indoor humidity, $\varphi_5$, by acting on the by-pass damper D, $\beta$.
> - The heating coil is not operating, $\dot{Q}_{sHC}=0$.
> - The mass flow rate of supply air, $\dot{m}$, is given and constant (CAV system).
> - The supply air temperature, $\theta_4 = \theta_3$, is not controlled.

*Notes on widgets*:
- `value`represents the controlled variable (i.e. φ); `sp`is its set-point value.
- Acting on the set point of indoor humidity, $\varphi_{5, sp}$ = `φ5sp`, has no effect (the heating coil does not operate).

In [6]:
cool4 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool4.VBP_wd, value='φ5', sp=(0.3, 0.5, 0.01), θo=(26, 34), φo=(0.4, 1),
            θ5sp=(20, 28), φ5sp=(0.4, 0.8, 0.01),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500))

interactive(children=(Text(value='φ5', description='value'), FloatSlider(value=0.5, description='sp', max=0.5,…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

### 6.1.2 Control of indoor and supply air temperatures in CAV systems

Controlling the supply air temperature, $\theta_4$, by acting on the by-pass, $\beta$, has **no soultion**.

Explanation:

In the sensible heat balace of the thermal zone [TZ],
$$\dot{m}c(\theta_4 - \theta_5) + \dot{Q}_{s, TZ} = 0$$

all valiables (i.e. $\dot{m}, \theta_4, \theta_5$ and $\dot{Q}_{s, TZ}$) are imposed. There is no way to control the supply air temperature, $\theta_4$, because the supply air mass flow rate, $\dot{m}$, is fixed in a CAV system. Given $\theta_5$ and $\dot{Q}_{sTZ}$:
- either $\dot{m}$ controls $\theta_4$,
- or $\theta_4$ controls $\dot{m}$.

In [7]:
# Executing the following code will generate an error: no solution for β
# cool = cc.MxCcRhTzBl(parameters, inputs)
# θ4sp = 11.77
# m = 3.162
# cool.actual[0] = m
# cool.VBP_wd('θS', θ4sp, θo, φo, θ5sp, φ5sp, mi, UA, QsBL, QlBL)

## 6.2 Mass-flow control in variable air volume (VAV) systems

### 6.2.1 Control of indoor air temperature & humidity in VAV systems without reheating

<img src="Figures/VAV_θ5_φ5.png" alt="VAV_θ5_φ5" style="width: 600px;"/>

>  VAV system controling: the indoor temperature, $\theta_5$, by acting on the cooling coil [CC], $\dot{Q}_{tCC}$, and the indoor air humidity, $\varphi_5$, by acting on the supply air mass flow rate, $\dot{m}$.
> - The heating coil is not operating, $\dot{Q}_{sHC}=0$.


In [8]:
cool3 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool3.VAV_wd, value='φ5', sp=(0.4, 0.5, 0.05), θo=(26, 34), φo=(0.4, 1),
            θ5sp=(20, 28), φ5sp=(0.4, 0.8),
            mi=(0.7, 3, 0.1), UA=(500, 800, 10), QsBL=(0, 60_000, 500), QlBL=(0, 20_000, 500))

interactive(children=(Text(value='φ5', description='value'), FloatSlider(value=0.5, description='sp', max=0.5,…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

### 6.2.2 Control of indoor and supply air temperatures in VAV systems without reheating

<img src="Figures/VAV_θ4_θ5.png" alt="VAV_θ4_θ5" style="width: 600px;"/>

> VAV system controling: indoor temperature, $\theta_5$, by acting on the cooling coil [CC], $\dot{Q}_{t,CC}$, and the supply air temperature, $\theta_4$, by acting on the supply air mass flow rate, $\dot{m}$.
> - The heating coil is not operating, $\dot{Q}_{sHC}=0$.

In [9]:
cool6 = cc.MxCcRhTzBl(parameters, inputs)
Kw = 0
cool6.actual[4] = Kw
wd.interact(cool6.VAV_wd, value='θ4', sp=(14, 21), θo=(26, 34), φo=(0.4, 1),
            θ5sp=(20, 28), φ5sp=(0.4, 0.8),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), QsBL=(0, 60_000, 500), QlBL=(0, 20_000, 500))

interactive(children=(Text(value='θ4', description='value'), IntSlider(value=14, description='sp', max=21, min…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

### 6.2.3 Control of indoor air temperature & humidity and of supply air temperature in VAV systems with reheating

<img src="Figures/VAV_θ4_θ5_φ5.png" alt="VAV_θ4_θ5_φ5" style="width: 600px;"/>

>  VAV system controling: indoor temperature, $\theta_5$, by acting on the cooling coil [CC], $\dot{Q}_{tCC}$,  indoor humidity, by acting on the heating coil [HC], $\dot{Q}_{s,HC}$, and supply air temperature, $\theta_4$, by acting on the supply air mass flow rate, $\dot{m}$.

In [10]:
cool6 = cc.MxCcRhTzBl(parameters, inputs)
Kw = 1e10
cool6.actual[4] = Kw
wd.interact(cool6.VAV_wd, value='θ4', sp=(14, 21), θo=(28, 36), φo=(0.4, 1),
            θ5sp=(22, 26), φ5sp=(0.4, 0.8),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), QsBL=(0, 60_000, 500), QlBL=(0, 20_000, 500))

interactive(children=(Text(value='θ4', description='value'), IntSlider(value=14, description='sp', max=21, min…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>